### Section A

#### Question A1
 - The number of lines is 1862234
 - Both languages have 1862234 lines
 - The number of partitions is 9
 
#### Question A2
 - Line counts still match after pre-processing (1862234)
 
#### Question A3
 - The most common in english are:
 [(3505228, 'the'),
 (2440596, ','),
 (1798163, '.'),
 (1662136, 'of'),
 (1543797, 'to'),
 (1319061, 'and'),
 (1089015, 'in'),
 (839170, 'that'),
 (774986, 'a'),
 (774949, 'is')]
 
  - The most common in swedish are:
  [(1849014, '.'),
 (1709915, 'att'),
 (1477733, ','),
 (1350825, 'och'),
 (1054362, 'i'),
 (952972, 'det'),
 (917581, 'som'),
 (914944, 'för'),
 (740761, 'av'),
 (701844, 'är')]
 

In [1]:
# A1
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://haol-spark:7077") \
        .appName("Question_A1")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
eng = spark_context.textFile("/home/ubuntu/LDSA/ass_2/swe_eng/europarl-v7.sv-en.en")
swe = spark_context.textFile("/home/ubuntu/LDSA/ass_2/swe_eng/europarl-v7.sv-en.sv")

In [3]:
# Count number of lines
eng.count()

1862234

In [4]:
swe.count()

1862234

In [5]:
# Take the first number of elements
eng.take(2)

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.']

In [6]:
swe.take(2)

['Återupptagande av sessionen',
 'Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december. Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester.']

In [7]:
eng.getNumPartitions()

9

In [8]:
swe.getNumPartitions()

9

In [9]:
# A2
# Return a RDD created by piping elements to a forked external process (perl tokenizer script).
engPiped = eng.pipe("/home/ubuntu/LDSA/ass_2/tools/tokenizer.perl -l en")
swePiped = swe.pipe("/home/ubuntu/LDSA/ass_2/tools/tokenizer.perl -l sv")

In [10]:
# Take the first number of elements
engPiped.take(2)

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999 , and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .']

In [11]:
swePiped.take(2)

['Återupptagande av sessionen',
 'Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december . Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester .']

In [12]:
engPiped.count()

1862234

In [13]:
swePiped.count()

1862234

In [15]:
engLowCase = engPiped.map(lambda line: line.lower().split("/n"))

In [16]:
sweLowCase = swePiped.map(lambda line: line.lower().split("/n"))

In [17]:
engLowCase.take(1)

[['resumption of the session']]

In [18]:
sweLowCase.take(1)

[['återupptagande av sessionen']]

In [19]:
engLowCase.count()

1862234

In [20]:
sweLowCase.count()

1862234

In [21]:
engWords = engLowCase.flatMap(lambda ls: ''.join(ls).split(" "))
sweWords = sweLowCase.flatMap(lambda ls: ''.join(ls).split(" "))

In [24]:
engWordsMapped = engWords.map(lambda x: (x,1))\
.groupByKey()
sweWordsMapped = sweWords.map(lambda x: (x,1))\
.groupByKey()

In [25]:
engWordsFreq = engWordsMapped.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)
sweWordsFreq = sweWordsMapped.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)

In [26]:
engWordsFreq.take(10)

[(3505228, 'the'),
 (2440596, ','),
 (1798163, '.'),
 (1662136, 'of'),
 (1543797, 'to'),
 (1319061, 'and'),
 (1089015, 'in'),
 (839170, 'that'),
 (774986, 'a'),
 (774949, 'is')]

In [27]:
sweWordsFreq.take(10)

[(1849014, '.'),
 (1709915, 'att'),
 (1477733, ','),
 (1350825, 'och'),
 (1054362, 'i'),
 (952972, 'det'),
 (917581, 'som'),
 (914944, 'för'),
 (740761, 'av'),
 (701844, 'är')]

In [28]:
# release the cores for another application!
spark_context.stop()